In [26]:
using GLMakie, LinearAlgebra, CUDA

# TODO: Try CUDA computation
# TODO: Add sliders for max force, max speed and perception radious
# TODO: Maybeeeeee try make .exe 

# Include file with movement algorithm
include("boids.jl")

# Make boids
axis_limit = 1500;
generation_area = 300
numBoids = 500
flock = [Boid(i, rand(-generation_area:generation_area, 3), randn(3).*3, randn(3).*2) for i=1:numBoids]
points = Observable([Point3f(boid.position) for boid in flock])

# Create figure
fig = Figure()

# Create 3d axis
ax = Axis3(fig[1, 1], aspect = (1, 1, 1), title = "Flocking simulation")
ax.perspectiveness = 0.3

# Set axis limits
limits!(ax, -axis_limit, axis_limit, -axis_limit, axis_limit, -axis_limit, axis_limit)

# Create sliders layout
lsgrid = labelslidergrid!(fig,
    ["align", "separation", "cohesion", "align perception", "separation perception", "cohesion perception"],
    [LinRange(0.1:0.01:2), LinRange(0.1:0.01:2), LinRange(0.1:0.01:2), LinRange(30:1:300), LinRange(30:1:300), LinRange(30:1:300)];
    formats = [x -> "$(round(x, digits = 2))"],
    labelkw = Dict([(:textsize, 20)]),
    sliderkw = Dict([(:linewidth, 10)]),
    valuekw = Dict([(:textsize, 20)])
)

# Create sliders grid
sl_sublayout = GridLayout(height=30)
fig[2, 1] = sl_sublayout
fig[2, 1] = lsgrid.layout

# Set initial parameters
set_close_to!(lsgrid.sliders[1], 0.7)
set_close_to!(lsgrid.sliders[2], 1.2)
set_close_to!(lsgrid.sliders[3], 0.6)
set_close_to!(lsgrid.sliders[4], 150)
set_close_to!(lsgrid.sliders[5], 100)
set_close_to!(lsgrid.sliders[6], 220)

# Update parameters values
alignValue = lsgrid.sliders[1].value
separationValue = lsgrid.sliders[2].value
cohesionValue = lsgrid.sliders[3].value
align_perception_radius = lsgrid.sliders[4].value
separate_perception_radius = lsgrid.sliders[5].value
cohese_perception_radius = lsgrid.sliders[6].value

# Create buttons grid
fig[3, 1] = buttongrid = GridLayout(height=100)

labels = ["Run / Pause", "Step", "Restart"]
button_count = 3

# Create buttons
buttons = buttongrid[3, 1:button_count] = [
    Button(fig, label=l, height=35, width=150, textsize=20)
    for l in labels
]

# Run button
isrunning = Observable(false)
on(buttons[1].clicks) do clicks; isrunning[] = !isrunning[]; end

on(buttons[1].clicks) do clicks
    @async while isrunning[]
        for boid in flock
            fly!(boid, flock, alignValue[], separationValue[], cohesionValue[], axis_limit[], align_perception_radius[], separate_perception_radius[], cohese_perception_radius[])
        end
        
        points[] = [Point3f(boid.position) for boid in flock]
        sleep(0.01)
    end
end

# Step button
on(buttons[2].clicks) do click
    for boid in flock
        fly!(boid, flock, alignValue[], separationValue[], cohesionValue[], axis_limit[], align_perception_radius[], separate_perception_radius[], cohese_perception_radius[])
    end
    
    points[] = [Point3f(boid.position) for boid in flock]
    sleep(0.01)
end

# Reset button
on(buttons[3].clicks) do click
    for boid in flock
        boid.position = rand(-generation_area:generation_area, 3)
    end
    points[] = [Point3f(boid.position) for boid in flock]
    sleep(0.01)
end

scatter!(ax, points, markersize=20, color=:black)
display(fig)

GLMakie.Screen(...)

In [44]:
using CUDA
CUDA.versioninfo()

CUDA toolkit 11.7, artifact installation
NVIDIA driver 512.59.0, for CUDA 11.6
CUDA driver 11.6

Libraries: 
- CUBLAS: 11.10.1
- CURAND: 10.2.10
- CUFFT: 10.7.1
- CUSOLVER: 11.3.5
- CUSPARSE: 11.7.3
- CUPTI: 17.0.0
- NVML: 11.0.0+512.59
- CUDNN: 8.30.2 (for CUDA 11.5.0)
- CUTENSOR: 1.4.0 (for CUDA 11.5.0)

Toolchain:
- Julia: 1.7.2
- LLVM: 12.0.1
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0
- Device capability support: sm_35, sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80

1 device:
  0: NVIDIA GeForce RTX 2060 (sm_75, 5.101 GiB / 6.000 GiB available)
